In [3]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import bs4

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Datenquellen laden

In [7]:
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()
print(len(docs), "Dokument(e) geladen")
print(docs[0].page_content[:400])

1 Dokument(e) geladen


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, e


### Texte in Chunks zerlegen

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    add_start_index=True
)
splits = text_splitter.split_documents(docs)
print(f"{len(splits)} Chunks erzeugt")

84 Chunks erzeugt


### Embedding in Chroma

In [9]:
embeddings = OllamaEmbeddings(model="all-minilm:33m")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="lilian_agents",
    persist_directory="./chroma_db"
)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":6})

### Test des Retrievers

In [10]:
query = "What are the approaches to Task Decomposition?"
results = retriever.invoke(query)

for i, doc in enumerate(results[:3], 1):
    print(f"\n--- Treffer {i} ---\n")
    print(doc.page_content[:350])


--- Treffer 1 ---

Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-tim

--- Treffer 2 ---

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a cl

--- Treffer 3 ---

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over pa

### LLM und Prompt

In [12]:
llm = OllamaLLM(model="llama3.1:latest")

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are a concise assistant. 
Use only the provided context to answer the question.
If the answer is not in the context, say you don't know.

Question:
{question}

Context:
{context}

Answer:"""
)

### RAG-Chain bauen

In [13]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Frage stellen

In [14]:
for chunk in rag_chain.stream("What is task decomposition in LLM agents?"):
    print(chunk, end="", flush=True)

Task decomposition in LLM agents refers to breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. According to the provided context, task decomposition is achieved through techniques such as Chain of Thought (CoT), which instructs the model to "think step by step" and decompose hard tasks into simpler steps.